In [1]:
%load_ext autoreload
%autoreload 2

from mast3r.datasets.pointodyssey import PointOdyssey
from mast3r.datasets.kubric import Kubric

In [41]:
dataset_p = PointOdyssey(
    root="data/point_odyssey",
    split="sample",
    strides=[1, 2, 4, 8, 16],
    clip_step=2,
    quick=False,
    verbose=True,
    aug_crop="auto",
    resolution=[(512, 256), (512, 384), (512, 336)],
    n_corres=512,
)

['data/point_odyssey/sample/r4_new_f/', 'data/point_odyssey/sample/scene_d78_0318_3rd/']
found 2 unique videos in data/point_odyssey (split=sample)
loading trajectories...
seq data/point_odyssey/sample/r4_new_f/
.....seq data/point_odyssey/sample/scene_d78_0318_3rd/
.....stride counts: {1: 1799, 2: 1797, 4: 1793, 8: 1785, 16: 1769}
resampled_num_clips_each_stride: [884, 1105, 1326, 1547, 1769]
collected 6631 clips of length 2 in data/point_odyssey (dset=sample)


In [5]:
dataset_k = Kubric(
    root="data/panning_movi_e_point_tracking",
    split="train",
    n_corres=128,
    strides=[1, 2, 3, 4, 5, 6, 7, 8],
    # clip_step=2,
    # quick=False,
    # verbose=True,
    aug_crop="auto",
    resolution=[(256, 256), (512, 384), (512, 336)],
    # n_corres=512,
)

found 20 unique videos in data/panning_movi_e_point_tracking (split=train)
loading trajectories...
stride counts: {1: 460, 2: 440, 3: 420, 4: 400, 5: 380, 6: 360, 7: 340, 8: 320}
resampled_num_clips_each_stride: [160, 182, 205, 228, 251, 274, 297, 320]
collected 1917 clips of length 2 in data/panning_movi_e_point_tracking (dset=train)


In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

d = dataset_k[(1000, 0)]
view1, view2 = d

img1 = view1["img"].permute(1, 2, 0).numpy() * 0.5 + 0.5
img2 = view2["img"].permute(1, 2, 0).numpy() * 0.5 + 0.5

viz_img = np.concatenate([img1, img2], axis=1)

corres1 = view1["corres"][view1["valid_corres"]]
corres2 = view2["corres"][view2["valid_corres"]]

plt.figure()
plt.imshow(viz_img)
plt.scatter(corres1[:, 0], corres1[:, 1], c="g", s=1)
plt.scatter(corres2[:, 0] + img1.shape[1], corres2[:, 1], c="r", s=1)
plt.plot(
    [corres1[:, 0], corres2[:, 0] + img1.shape[1]],
    [corres1[:, 1], corres2[:, 1]],
    c="r",
    lw=0.5,
)

(np.int32(3), np.int32(244), np.int32(0), np.int32(255))

In [251]:
import open3d as o3d


pts1 = view1["pts3d"].reshape(-1, 3).astype(np.float64)
pts2 = view2["pts3d"].reshape(-1, 3).astype(np.float64)
colors1 = (
    view1["img"].permute(1, 2, 0).contiguous().reshape(-1, 3).numpy().astype(np.float64)
) * 0.5 + 0.5
colors2 = (
    view2["img"].permute(1, 2, 0).contiguous().reshape(-1, 3).numpy().astype(np.float64)
) * 0.5 + 0.5

# tint colors 1 with red
colors1 = colors1 * np.array([1, 0.5, 0.5])
# tint colors 2 with blue
colors2 = colors2 * np.array([0.5, 0.5, 1])

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pts1)
pcd1.colors = o3d.utility.Vector3dVector(colors1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pts2)
pcd2.colors = o3d.utility.Vector3dVector(colors2)

o3d.visualization.draw_geometries([pcd1, pcd2])